In [1]:
import pandas as pd
import numpy as np

1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [2]:
import csv
my_list=np.random.randint(low=0,high=1000,size=20) ##creating an array of 20 cells with values from 0 to 1000
out_1="data_int.txt" 
with open(out_1, "w") as out_file:
    out_file.write(np.array2string(my_list)+"\n") ##writing the list into the file
!cat data_int.txt

matrix=np.random.rand(5,5)*1000 ## the matrix has the higher value of 1000 as the previous array
out_2="data_float.txt"
with open(out_2, "w") as out_file:
    out_file.write(np.array2string(matrix)+"\n") ##same procedure as before
!cat data_float.txt

out_csv = "all_data.csv"

with open(out_1,"r") as input_file:  ##here we can choose which file to transfer into a csv
    with open(out_csv, "w") as output_file:
        w=csv.writer(output_file)
        for line in csv.reader(input_file, delimiter=" "): ##the delimiter is a blank space
            while(" " in line):
                line.remove(" ") ##remove that
            for (i,j) in zip(line,range(0,len(line))): 
                line[j]=i.replace("[","").replace("]","") ##remove the square brackets and replace them with empty char
            w.writerow(line)
            
            

                
    
    


[997 277 601 238 896 306 152  98 431 404 153 454 410 502 316 303 462 579
 952 121]
[[ 59.28630088 291.84574411 990.08070287  47.61890808 997.17189692]
 [293.28246219 797.38848848 176.7906323  455.33899868 441.7334972 ]
 [621.2955568  275.46490744 706.83825475 507.87892923 799.23030343]
 [983.8311946  520.90808073 434.74031458 721.13251013 422.74239745]
 [760.46631497 333.65457628 972.47172328   6.77603126 921.48653241]]


2\. **JSON files**

Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [3]:
#!wget https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

In [4]:
import json
dataset = json.load(open("user_data.json"))
american_ex=[x for x in dataset if x["CreditCardType"] == "American Express"] ##filter by credit card type
fieldnames = ['ID', 'JobTitle', 'EmailAddress', 'FirstNameLastName','CreditCard','CreditCardType'] ##list of all field names
with open("output_credit_card.csv", "w",newline="") as file:
    w = csv.DictWriter(file, fieldnames=fieldnames) ##write into a csv
    w.writeheader()
    w.writerows(american_ex)


3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [5]:
#!wget https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

In [6]:
dataset = pd.read_csv("mushrooms_categorized.csv")
feature_mean=dataset.groupby("class").mean() ##using groupby to compute the mean for each class
print("The mean of the features for each class are:\n", feature_mean)
import json
feature_mean.to_json("mushrooms_categorized.json",orient="index") ##saving the file into a .json




The mean of the features for each class are:
        cap-shape  cap-surface  cap-color   bruises      odor  gill-attachment  \
class                                                                           
0       3.266160     1.615970   4.581749  0.653992  4.334601         0.954373   
1       3.436159     2.055158   4.421859  0.159346  3.940756         0.995403   

       gill-spacing  gill-size  gill-color  stalk-shape  ...  \
class                                                    ...   
0          0.285171   0.068441    6.622624     0.615970  ...   
1          0.028601   0.567926    2.863636     0.514811  ...   

       stalk-surface-below-ring  stalk-color-above-ring  \
class                                                     
0                      1.798479                6.098859   
1                      1.394280                5.512768   

       stalk-color-below-ring  veil-type  veil-color  ring-number  ring-type  \
class                                                  

4\. **Reading the credit card numbers**

Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [7]:
#!wget https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

In [8]:
with open("credit_card.dat","rb") as f: 
    file_lines= f.readlines() ##reading the lines
    for line in file_lines:
        my_line=line.decode().replace("\n","") ##avoiding \n
        card_num="" ##empty string
        for i in range(0,len(my_line),6):
            char_card=my_line[i:i+6]
            number=int(char_card,2) 
            card_num=card_num+str(chr(number)) ##procedure suggested in the hint
        print(card_num) ## printing then the result,
                                    

7648 5673 3775 2271

3257 8247 3354 2266

2722 0001 4011 6652

0661 3063 3742 3150

0432 1608 1462 4742

5827 2027 8785 7303

5774 8528 2087 1117

8140 1210 6352 2845

5764 1133 7301 7100

6456 1737 4126 6726

1228 8631 7382 0000

7051 0160 5374 3166

0618 3587 1630 6376

1545 5454 7444 5636

6735 3116 3202 6834

7287 5011 1547 8413

7033 2607 3328 4200

2568 5244 1874 5024

1684 2253 7570 7118

0672 2576 0575 6631

6332 8353 8787 1340

1813 3361 1175 4211

2477 6450 8840 2368

5512 3505 2563 1326

3083 7882 0621 0025

4521 5148 8045 0334

7563 3654 8713 5787

8324 2664 0476 5561

0565 2504 7168 3510

5107 5507 1767 0738

2462 1821 2448 1443

2788 0638 6861 6554

5851 5873 5474 0547

0670 1004 4013 2655

5874 5506 3048 0806

2805 5401 8462 1260

5083 8406 6310 1862

1076 1445 3013 2266

8440 4804 4844 5277

4758 6141 0686 1387

7586 0675 0315 2568

2544 1258 7432 5165

3474 5023 4434 5626

1410 0270 0434 5086

7315 4446 1104 4215

0224 7742 8300 0266

0170 2700 3145 0640

2006 2437 805

5\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [10]:
#from IPython.display import Image
#Image("images/data_format.png")

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, ``pack'' the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_OSEMN.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?

In [12]:
import struct
file= "./data/data_000637.txt" ##name of the data file


dataset = pd.read_csv(file, nrows=10) #reading first 10 rows


output= "binary_file.dat" 

with open(output, 'wb') as bin_file: ##using wb as parameter in order to write in binary
    for line in dataset.values : 
        word = (line[5] << 0 ) 
        word = word+ (line[4] << 5 )
        word = word+(line[3] << 17) 
        word = word+(line[2] << 49)  ##standard specified in the image
        word = word+(line[1] << 58)
        word = word+(line[0] << 62)

    bin_file.write( struct.pack("<q", word) ) ##hint

## the .dat file is smaller than the .txt file